In [1]:
################# Question b-i: ModifiedApriori ###############################
from itertools import combinations
# Modified Apriori function to generate frequent itemsets up to a specified maximum size N
def ModifiedApriori(transactions, min_support, max_itemset_size):
    # Step 1: Count item frequencies in transactions to get 1-item frequent sets
    item_counts = {}
    for transaction in transactions:
        for item in transaction:
            item_counts[frozenset([item])] = item_counts.get(frozenset([item]), 0) + 1

    # Filter 1-itemsets by min_support
    num_transactions = len(transactions)
    min_support_count = min_support * num_transactions
    frequent_itemsets = {itemset: count for itemset, count in item_counts.items() if count >= min_support_count}

    # Iteratively generate higher-order frequent itemsets up to max_itemset_size
    k = 2
    current_freq_itemsets = list(frequent_itemsets.keys())
    
    while k <= max_itemset_size and current_freq_itemsets:
        # Generate candidate k-itemsets from (k-1)-itemsets
        candidate_k_itemsets = set(
            [frozenset(i.union(j)) for i in current_freq_itemsets for j in current_freq_itemsets if len(i.union(j)) == k]
        )
        
        # Count support for each candidate k-itemset
        itemset_counts = {candidate: 0 for candidate in candidate_k_itemsets}
        for transaction in transactions:
            transaction_frozenset = frozenset(transaction)
            for candidate in candidate_k_itemsets:
                if candidate.issubset(transaction_frozenset):
                    itemset_counts[candidate] += 1
        
        # Filter candidates by min_support
        current_freq_itemsets = [itemset for itemset, count in itemset_counts.items() if count >= min_support_count]
        frequent_itemsets.update({itemset: itemset_counts[itemset] for itemset in current_freq_itemsets})
        
        k += 1

    return frequent_itemsets

# Load and preprocess the dataset
import pandas as pd

# Load the dataset
data = pd.read_csv("HW3_Music_Playlists")

# Convert each row to a list of items (songs) for transactions
transactions = data.apply(lambda row: frozenset(row.dropna().values), axis=1).tolist()

# Define parameters
min_support = 0.004  # 0.4%
max_itemset_size = 5

# Run ModifiedApriori
frequent_itemsets = ModifiedApriori(transactions, min_support, max_itemset_size)

# Display the frequent itemsets
frequent_itemsets

/tmp/ipykernel_33947/3000775480.py:48: DtypeWarning: Columns (36) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("/home/vineel/Desktop/data mining/HW's/HW3_Music_Playlists.csv")


{frozenset({' All The Way Up'}): 1040,
 frozenset({' How to Save a Life'}): 246,
 frozenset({' Love Faces'}): 152,
 frozenset({' Walk'}): 1370,
 frozenset({' My Cocaine'}): 225,
 frozenset({' oui'}): 3253,
 frozenset({' Take A Back Road'}): 809,
 frozenset({' Touch My Body'}): 2544,
 frozenset({' The Devil Went Down to Georgia'}): 2825,
 frozenset({' Best Song Ever'}): 540,
 frozenset({'Bombs of My Love'}): 125,
 frozenset({' bird'}): 1642,
 frozenset({' Puzzles'}): 180,
 frozenset({' Cheerleader - Felix Jaehn Remix Radio Edit'}): 3546,
 frozenset({' Norwegian Wood (This Bird Has Flown) - Remastered'}): 1937,
 frozenset({' Timber'}): 745,
 frozenset({' The Joker'}): 231,
 frozenset({' Blah Blah Blah'}): 815,
 frozenset({" Buy U a Drank (Shawty Snappin')"}): 1387,
 frozenset({' I Need You More'}): 3541,
 frozenset({' Prayer'}): 200,
 frozenset({' Vanished'}): 480,
 frozenset({' My Own Prison'}): 374,
 frozenset({' Walked In'}): 431,
 frozenset({' Fireflies'}): 260,
 frozenset({' My Girl

In [6]:
################# Question b-ii: RuleGeneration ###############################
# Helper function to calculate confidence
def calculate_confidence(frequent_itemset, antecedent, support_data):
    antecedent_support = support_data[antecedent]
    if antecedent_support > 0:
        return support_data[frequent_itemset] / antecedent_support
    return 0
# Helper function to generate k-item consequents
def generate_k_itemsets(itemsets, k):
    """Generate k-item consequents from the given itemsets."""
    return [frozenset(x) for x in combinations(set().union(*itemsets), k)]
# Rule generation following Algorithm 4.2 with single-item consequents initially
def RuleGeneration(frequent_itemsets, min_confidence):
    rules = []  # Store the generated rules
    support_data = {itemset: support for itemset, support in frequent_itemsets.items()}
    
    # For each frequent itemset with size >= 2
    for itemset in frequent_itemsets:
        if len(itemset) >= 2:
            # Generate initial 1-item consequents
            consequents = [frozenset([item]) for item in itemset]
            # Generate rules with recursive helper function
            rules += ap_genrules(itemset, consequents, support_data, min_confidence)
    
    return rules

# Recursive function to expand consequents and generate rules
def ap_genrules(frequent_itemset, consequents, support_data, min_confidence):
    rules = []
    m = len(consequents[0])  # Size of consequent itemsets (starts with 1)

    # Calculate and add valid rules with confidence above min_confidence
    valid_consequents = []
    for consequent in consequents:
        antecedent = frequent_itemset - consequent
        confidence = calculate_confidence(frequent_itemset, antecedent, support_data)
        
        if confidence >= min_confidence:
            # Add rule to rules list
            rules.append((antecedent, consequent, confidence))
            valid_consequents.append(consequent)
    
    # Expand the consequents if possible
    if len(frequent_itemset) > m + 1 and valid_consequents:
        # Generate (m+1)-item consequents for the next recursion level
        expanded_consequents = generate_k_itemsets(valid_consequents, m + 1)
        # Recursively generate rules for larger consequents
        rules += ap_genrules(frequent_itemset, expanded_consequents, support_data, min_confidence)

    return rules


In [7]:

################# Question b-iii and c: Running RuleGeneration and Interpretation ###############################

# Minimum confidence threshold
min_confidence = 0.005  # 0.5% threshold

# Generate association rules from frequent itemsets
association_rules = RuleGeneration(frequent_itemsets, min_confidence)

# Sort rules by confidence in descending order and display top K
K = 5  # Top K rules
sorted_rules = sorted(association_rules, key=lambda x: x[2], reverse=True)[:K]

# Display the top K rules with interpretations
for antecedent, consequent, confidence in sorted_rules:
    print(f"Rule: {set(antecedent)} -> {set(consequent)}, confidence: {confidence:.2f}")
    # Interpretation for each rule
    print(f"Interpretation: Users who listen to {set(antecedent)} are likely to enjoy {set(consequent)} as well. Confidence: {confidence:.2f}")

Rule: {' On My Own'} -> {' Bebe Rexha & Afrojack)'}, confidence: 0.89
Interpretation: Users who listen to {' On My Own'} are likely to enjoy {' Bebe Rexha & Afrojack)'} as well. Confidence: 0.89
Rule: {' I Need You More', ' Cheap Thrills'} -> {" Don't Deserve You"}, confidence: 0.79
Interpretation: Users who listen to {' I Need You More', ' Cheap Thrills'} are likely to enjoy {" Don't Deserve You"} as well. Confidence: 0.79
Rule: {' I Need You More', ' Legend'} -> {' The Devil Went Down to Georgia'}, confidence: 0.73
Interpretation: Users who listen to {' I Need You More', ' Legend'} are likely to enjoy {' The Devil Went Down to Georgia'} as well. Confidence: 0.73
Rule: {' Maliblue'} -> {' Cheerleader - Felix Jaehn Remix Radio Edit'}, confidence: 0.60
Interpretation: Users who listen to {' Maliblue'} are likely to enjoy {' Cheerleader - Felix Jaehn Remix Radio Edit'} as well. Confidence: 0.60
Rule: {' Cheap Thrills'} -> {" Don't Deserve You"}, confidence: 0.59
Interpretation: Users who